## Importo paquetes:

In [10]:
import pandas as pd
import json
import os
import glob
from pypdf import PdfReader
import re
import tqdm
import pickle
from tqdm import tqdm

from langchain.chat_models import ChatOpenAI

from typing import List, Optional
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Funciones

In [43]:
def bill_cleaner(path):

    """
    Función que devuelve el texto procesado de una factura.

    Input:
        - path(str): Ruta de la factura.pdf
    
    Output:
        - texto_limpio (str)
    
    """

    factura = PdfReader(path)
    
    texto_factura = ""
    for pagina in factura.pages:
        texto_factura += pagina.extract_text()
    
    # Elimino saltos de linea
    texto_limpio = re.sub(r"\s+", " ", texto_factura).strip()
    
    # Elimino puntos
    texto_limpio = re.sub(r"\.+", "", texto_limpio)
    
    # Elimino espacios multiples
    texto_limpio = re.sub(r"\s+", " ", texto_limpio)

    return texto_limpio

In [44]:
def information_extractor(path, openai_api_key):

    """
    Función que extrae la información requerida de una factura de luz guardandola en formato JSON.

    Input:
        - path(str): Ruta donde se alojan las factura en formato PDF
        - openai_api_key(str): La api key de OPENAI
    
    Output:
        - archivo JSON con la información requerida de la factura.
    
    """
    

    # Defino el LLM:
    llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    openai_api_key= openai_api_key)

    # Cargo el esquema:
    with open("utils/schema.pkl", "rb") as f:
        schema = pickle.load(f)

    # Cargo cadena
    chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
    
    
    
    # Busco las facturas en PDF en la ruta proporcionada:
    
    facturas = glob.glob(os.path.join(path, "*.pdf"))

    # Obtengo la información requerida de cada factura:
    
    if not os.path.exists("extracted_information"):
        os.makedirs("extracted_information")
    
    barra_progreso = tqdm(total= len(facturas), desc="Progreso", unit="archivo")
    
    for path_factura in facturas:
        file_name = path_factura.split("\\")[1].split(".")[0] + ".json"
        information_path = "extracted_information\\"

        ###################################### Limpieza texto ######################################
        
        texto_factura = bill_cleaner(path_factura)

        ###################################### Extracción de información requerida ######################################
        
        informacion_factura = chain.invoke(input= texto_factura)["text"]["data"]["informacion_factura"][0]

        ###################################### Guardo en formato JSON la informacion ######################################
        with open(information_path + file_name, "w") as json_file:
            json.dump(informacion_factura, json_file, indent=4)

        
        barra_progreso.update(1)

    barra_progreso.close()

# Inicializo variables principales:

- data_path: Es la carperta donde se encuentran las facturas en pdf
- openai_api_key: La clave de la API de OPENAI en el archivo .env

In [45]:
openai_api_key= os.getenv("OPENAI_API_KEY")
data_path = "data/"

# Extraigo información de las facturas

In [25]:
information_extractor(data_path, openai_api_key)


Progreso: 100%|██████████| 5/5 [06:24<00:00, 76.94s/archivo]

Progreso: 100%|██████████| 5/5 [00:31<00:00,  6.35s/archivo]


## Compruebo

In [21]:
data_path = "data\\"
extracted_info_path = "extracted_information\\"
data = pd.read_csv(data_path + "df_finetuning.csv")

In [32]:
factura_0 = data.loc[100].output.replace("'", '"')
json.loads(factura_0)

{'nombre_cliente': 'Hera Alcocer Zuleta',
 'dni_cliente': '72165903E',
 'calle_cliente': 'Calle de la Amapola',
 'cp_cliente': '19276',
 'población_cliente': 'Hijes',
 'provincia_cliente': 'Guadalajara',
 'nombre_comercializadora': 'IE2 INNOVACION, S.L.U.',
 'cif_comercializadora': 'B39829197',
 'dirección_comercializadora': 'C/ ISABEL TORRES, Nº25',
 'cp_comercializadora': '39011',
 'población_comercializadora': 'SANTANDER',
 'provincia_comercializadora': 'Cantabria',
 'número_factura': 'TO1016699250',
 'inicio_periodo': '18.10.2008',
 'fin_periodo': '17.11.2008',
 'importe_factura': '82,43',
 'fecha_cargo': '22.11.2008',
 'consumo_periodo': 256,
 'potencia_contratada': '4,565'}

In [33]:
with open(extracted_info_path + f"factura_{100}.json", "r", encoding= "utf-8") as archivo:
        factura_0_test = json.load(archivo)

In [34]:
factura_0_test

{'nombre_cliente': 'Hera Alcocer Zuleta',
 'dni_cliente': '72165903E',
 'calle_cliente': 'Calle de la Amapola',
 'cp_cliente': '19276',
 'población_cliente': 'Hijes',
 'provincia_cliente': 'Guadalajara',
 'nombre_comercializadora': 'IE2 INNOVACION, SLU',
 'cif_comercializadora': 'B39829197',
 'dirección_comercializadora': 'C/ ISABEL TORRES, Nº25',
 'cp_comercializadora': '39011',
 'población_comercializadora': 'SANTANDER',
 'provincia_comercializadora': 'Cantabria',
 'número_factura': 'TO1016699250',
 'inicio_periodo': '18.10.2008',
 'fin_periodo': '17.11.2008',
 'importe_factura': '82,43',
 'fecha_cargo': '22.11.2008',
 'consumo_periodo': '256',
 'potencia_contratada': '4,565'}